In [1]:
import numpy as np
import pymongo
import math
from collections import defaultdict
from functools import partial
import datetime
from dateutil import relativedelta

In [64]:
client = pymongo.MongoClient('localhost', 27017, maxPoolSize=100)
db = client["Stocks"]
stocks = db.list_collection_names()

In [65]:
def update_stocks_return(stocks, db):
    for stock in stocks:
        curr_db = db[stock]
        
        last_price = None
        last_id = None

        for thing in curr_db.find({'LAST_PRICE':{'$exists': True}}).sort([('Date', 1)]):
            curr_price = thing['LAST_PRICE']
            if last_price != None:
                last_return = math.log(curr_price/last_price)
                curr_db.update_one({"_id": last_id}, {'$set': {'LOG_RETURN': last_return}})

            last_price = curr_price
            last_id = thing['_id']